In [2]:
import os
import argparse
import math
import time
import ast
from copy import copy
from glob import glob

import pandas as pd
import numpy as np

import pyproj
from pyproj import Proj
import cv2
from shapely.ops import transform
import rioxarray
import geopandas as gpd

import ultralytics
from ultralytics import YOLO
#ultralytics.checks()

import concurrent
import concurrent.futures
from concurrent.futures.thread import ThreadPoolExecutor

In [3]:
import os
import argparse
import math
import time
import ast
from copy import copy
from glob import glob

import pandas as pd
import numpy as np
import pyproj 
from pyproj import Proj
import shapely
import geopandas as gpd
from shapely.ops import transform
from shapely.geometry import Point
import rioxarray

import ultralytics
from ultralytics import YOLO


def chunk_df(img_paths, num_chunks=10):
    # Calculate the number of rows per chunk
    rows_per_chunk = len(img_paths) // num_chunks
    df_chunks = [np.array(img_paths[i : i + rows_per_chunk]) for i in range(0, len(img_paths), rows_per_chunk)]
    return df_chunks


def tile_dimensions_and_utm_coords(tile_path): #used
    """ Obtain tile band, height and width and utm coordinates
    Args: tile_path(str): the path of the tile 
    Returns: 
    utmx(np array): the x utm coordinates corresponding with the tile coordinate convention (origin in upper left hand corner)
    utmy(np array): the y utm coordinates corresponding with the tile coordinate convention (origin in upper left hand corner)
    tile_band(int): the number of bands
    tile_height(int): the height of the tile (in pixels)
    tile_width(int): the width of the tile (in pixels)
    """
    ## Get tile locations
    da = rioxarray.open_rasterio(tile_path) ## Read the data
    # Compute the lon/lat coordinates with rasterio.warp.transform
    # lons, lats = np.meshgrid(da['x'], da['y'])
    tile_band, tile_height, tile_width = da.shape[0], da.shape[1], da.shape[2]
    utmx = np.array(da['x'])
    utmy = np.array(da['y'])
    crs =  str(rioxarray.open_rasterio(tile_path).rio.crs)
    return(utmx, utmy, crs, tile_band, tile_height, tile_width)
    del da


def process_results(results, tile_height, tile_width, item_dim):
    #xyxys = []
    bbox_pixel_coords_list = [] #xyxy coords with repsect to the tile
    conf_list = [] #probability
    class_name_list = [] #class name
    image_names_list = []
    tile_names_list = []
    lat_lons = []
    for result in results:
        boxes = result.boxes
        image_name = os.path.splitext(os.path.basename(result.path))[0]
        tile_name = image_name.rsplit("_",2)[0]
        if len(boxes) > 0: 
            #get valeus for all bounding boxes
            #class name
            class_name_list.extend([result.names[class_number] for class_number in boxes.cls.cpu().detach().tolist()])
            
            conf_list.extend(boxes.conf.cpu().detach().tolist())

            xyxy = boxes.xyxy.cpu().detach().numpy() - 1 #read xmin,ymin,xmax,ymax coordinates to memory as a numpy array
            xyxy = np.round(xyxy).astype(np.int32).tolist()  # round so that it can be used for utm to lonlat conversion, check if zero indexed
            image_names_list.extend([image_name]*len(xyxy)) # The index is a six-digit number like '000023'.
            tile_names_list.extend([tile_name]*len(xyxy)) # The index is a six-digit number like '000023'.
            #calculate the tile level pixel coordinates
            bbox_pixel_coords_list.extend([calculate_tile_level_bbox(image_name, box, item_dim,
                                                                     tile_width, tile_height) for box in xyxy])
        del boxes
    return pd.DataFrame({"confidence":conf_list, "class_name": class_name_list,
                       "image_names": image_names_list, "tile_names": tile_names_list, 
                         "bbox_pixel_coords": bbox_pixel_coords_list})#,  dtype=dtypes


def predict_process(img_paths, tile_height, tile_width, model, args):
    # obtain predictions over the dataframe
    results_df = pd.DataFrame({})
    num_chunks = len(img_paths)//50
    for df_chunk in chunk_df(img_paths, num_chunks=num_chunks):
        results = model.predict(df_chunk.tolist(), save=False, imgsz=args.imgsz)#, conf=0.5)
        #process_results(results, utmx, utmy, utm_proj, tile_height, tile_width, item_dim=args.imgsz)
        results_df = pd.concat([results_df, process_results(results, tile_height, tile_width, item_dim=args.imgsz)])
        del results, df_chunk
    return results_df


def calculate_tile_level_bbox(image_name, xyxy, item_dim, tile_width, tile_height):
    obj_xmin, obj_ymin, obj_xmax, obj_ymax = xyxy
    #identify rows and columns
    y, x = image_name.split("_")[-2:] #name of tif with the extension removed; y=row;x=col
    # Each chip xml goes from 1 - item_dim, specify the "0", or the end point of the last xml
    image_minx = int(x)*item_dim
    image_miny = int(y)*item_dim

    #add the bounding boxes
    obj_xmin = image_minx + obj_xmin
    obj_ymin = image_miny + obj_ymin
    obj_xmax = image_minx + obj_xmax
    obj_ymax = image_miny + obj_ymax
    
    # correct bboxes that extend past the bounds of the tile width/height
    if int(obj_xmin) >= tile_width:
        obj_xmin = tile_width - 1
    if int(obj_xmax) >= tile_width:
        obj_xmax = tile_width - 1
    if int(obj_ymin) >= tile_height:
        obj_ymin = tile_height - 1
    if int(obj_ymax) >= tile_height:
        obj_ymax = tile_height - 1
    
    return [obj_xmin, obj_ymin, obj_xmax, obj_ymax]


def transform_point_utm_to_wgs84(utm_proj, utm_xcoord, utm_ycoord): #used
    """ Convert a utm pair into a lat lon pair 
    Args: 
    utm_proj(str): the UTM string as the in term of EPSG code
    utmx(int): the x utm coordinate of a point
    utmy(int): the y utm coordinates of a point
    Returns: 
    (wgs84_pt.x, wgs84_pt.y): the 'EPSG:4326' x and y coordinates 
    """
    #https://gis.stackexchange.com/questions/127427/transforming-shapely-polygon-and-multipolygon-objects
    #get utm projection
    utm = pyproj.CRS(utm_proj)
    #get wgs84 proj
    wgs84 = pyproj.CRS('EPSG:4326')
    #specify utm point
    utm_pt = Point(utm_xcoord, utm_ycoord)
    #transform utm into wgs84 point
    project = pyproj.Transformer.from_crs(utm, wgs84, always_xy=True).transform
    wgs84_pt = transform(project, utm_pt)
    return wgs84_pt.x, wgs84_pt.y
    
    
def get_utm_coords(pixel_coords, utmx, utmy):
    minx, miny, maxx, maxy = pixel_coords
    return [utmx[minx], utmy[miny], utmx[maxx], utmy[maxy]]  
    
    
def get_lat_lon_coords(utm_coords, utm_proj):
    minx, miny, maxx, maxy = utm_coords
    #determine the lat/lon
    nw_lon, nw_lat = transform_point_utm_to_wgs84(utm_proj, minx, miny)
    se_lon, se_lat = transform_point_utm_to_wgs84(utm_proj, maxx, maxy) 
    return [nw_lon, nw_lat, se_lon, se_lat]

        
def merge_boxes(bbox1, bbox2): #used
    """ 
    Generate a bounding box that covers two bounding boxes
    Called in merge_algo
    Arg:
    bbox1(list): a list of the (xmin, ymin, xmax, ymax) coordinates for box 1 
    bbox2(list): a list of the (xmin, ymin, xmax, ymax) coordinates for box 2
    Returns:
    merged_bbox(list): a list of the (xmin, ymin, xmax, ymax) coordinates for the merged bbox

    """
    return [min(bbox1[0], bbox2[0]), 
            min(bbox1[1], bbox2[1]),
            max(bbox1[2], bbox2[2]),
            max(bbox1[3], bbox2[3])]


def calc_sim(bbox1, bbox2, dist_limit): #used
    """Determine the similarity of distances between bboxes to determine whether bboxes should be merged
    Computer a Matrix similarity of distances of the text and object
    Called in merge_algo
    Arg:
    bbox1(list): a list of the (xmin, ymin, xmax, ymax) coordinates for box 1 
    bbox2(list): a list of the (xmin, ymin, xmax, ymax) coordinates for box 2
    dist_list(int): the maximum threshold (pixel distance) to merge bounding boxes
    Returns:
    (bool): to indicate whether the bboxes should be merged 
    """

    # text: ymin, xmin, ymax, xmax
    # obj: ymin, xmin, ymax, xmax
    bbox1_xmin, bbox1_ymin, bbox1_xmax, bbox1_ymax = bbox1
    bbox2_xmin, bbox2_ymin, bbox2_xmax, bbox2_ymax = bbox2
    x_dist = min(abs(bbox2_xmin-bbox1_xmax), abs(bbox2_xmax-bbox1_xmin))
    y_dist = min(abs(bbox2_ymin-bbox1_ymax), abs(bbox2_ymax-bbox1_ymin))
        
    #define distance if one object is inside the other
    if (bbox2_xmin <= bbox1_xmin) and (bbox2_ymin <= bbox1_ymin) and (bbox2_xmax >= bbox1_xmax) and (bbox2_ymax >= bbox1_ymax):
        return True
    elif (bbox1_xmin <= bbox2_xmin) and (bbox1_ymin <= bbox2_ymin) and (bbox1_xmax >= bbox2_xmax) and (bbox1_ymax >= bbox2_ymax):
        return True
    #determine if both bboxes are close to each other in 1d, and equal or smaller length in the other
    elif (x_dist <= dist_limit) and (bbox1_ymin <= bbox2_ymin) and (bbox1_ymax >= bbox2_ymax): #bb1 bigger
        return True
    elif (x_dist <= dist_limit) and (bbox2_ymin <= bbox1_ymin) and (bbox2_ymax >= bbox1_ymax): #bb2 bigger
        return True
    elif (y_dist <= dist_limit) and (bbox1_xmin <= bbox2_xmin) and (bbox1_xmax >= bbox2_xmax): #bb1 bigger
        return True
    elif (y_dist <= dist_limit) and (bbox2_xmin <= bbox1_xmin) and (bbox2_xmax >= bbox1_xmax): #bb2 bigger
        return True
    else: 
        return False

    
def merge_predicted_bboxes(results_df, dist_limit = 5):
    class_names = results_df.class_name.to_list()
    bbox_pixel_coords = results_df.bbox_pixel_coords.to_list()
    confidences = results_df.confidence.to_list()
    tile_names = results_df.tile_names.to_list()
    merge_bools = [False] * len(class_names)
    for i, (conf1, class_name1, bbox1, tile_name1) in enumerate(zip(confidences, class_names, bbox_pixel_coords, tile_names)):
        for j, (conf2, class_name2, bbox2, tile_name2) in enumerate(zip(confidences, class_names, bbox_pixel_coords, tile_names)):
            if j <= i: #only consider the remaining bboxes
                continue
            # Create a new box if a distances is less than distance limit defined 
            merge_bool = calc_sim(bbox1, bbox2, dist_limit) 
            if merge_bool == True:
                # Create a new box  
                new_box = merge_boxes(bbox1, bbox2)   
                bbox_pixel_coords[i] = new_box
                #delete previous text boxes
                del bbox_pixel_coords[j]
                class_name_merge = np.unique([class_name1, class_name2])
                conf = [conf1, conf2]

                class_names[i] = class_name_merge
                confidences[i] = conf

                conf
                #delete previous text 
                del class_names[j],  confidences[j], tile_names[j]
    return pd.DataFrame({"confidence":confidences, "class_name": class_names,
                         "bbox_pixel_coords": bbox_pixel_coords, "tile_names": tile_names})#,  dtype=dtypes


def write(predictions, predictions_file_path):
    # remove file if it exists 
    if os.path.exists(predictions_file_path):
        predictions.to_parquet(predictions_file_path, engine='fastparquet', append=True)
    else:
        predictions.to_parquet(predictions_file_path, engine='fastparquet')
        

def calculate_diameter(bbox, resolution = 0.6): #used
    """ Calculate the diameter of a given bounding bbox (in Pascal Voc Format) for imagery of a given resolution
    Arg:
    bbox(list): a list of the (xmin, ymin, xmax, ymax) coordinates for box. Utm coordinates are provided as [nw_x_utm, se_y_utm, se_x_utm, nw_y_utm] to conform with Pascal Voc Format.
    resolution(float): the (gsd) resolution of the imagery
    Returns:
    (diameter): the diameter of the bbox of interest
    """
    obj_xmin, obj_ymin, obj_xmax, obj_ymax = bbox
    obj_width = obj_xmax - obj_xmin
    obj_height = obj_ymin - obj_ymax 
    diameter = min(obj_width, obj_height) * resolution #meter
    return diameter

        
def get_args_parse():
    parser = argparse.ArgumentParser("Predict on images")    
    parser.add_argument("--chunk_id",  type=int)
    parser.add_argument("--tile_dir", default="/work/csr33/images_for_predictions/naip_tiles", type=str)
    parser.add_argument("--tilename_chunks_path", default='/hpc/home/csr33/ast_object_detection/tilename_chunks.npz', type=str)
    parser.add_argument("--model_path", default="/work/csr33/object_detection/runs/detect/baseline_train/weights/best.pt", type=str)
    parser.add_argument("--prediction_dir", default="/work/csr33/images_for_predictions/predictions", type=str)
    parser.add_argument("--imgsz", default=640, type=int)
    parser.add_argument('--img_dir', type=str, default="/work/csr33/images_for_predictions/naip_imgs")
    parser.add_argument('--classification_threshold', type=float, default=0.5)

    args = parser.parse_args()
    return args


In [6]:

    os.chdir("/work/csr33/object_detection")
    #determine chunk-number   
    os.makedirs(args.prediction_dir, exist_ok=True)
    model = YOLO(args.model_path)  # custom trained model 
    
    # load a subset of the tile paths to predict on
    tile_paths = np.load(args.tilename_chunks_path)[str(args.chunk_id)][:1]
    tile_names = [os.path.splitext(os.path.basename(tile_path))[0] for tile_path in tile_paths]
    
    #intialize dataframes
    predict_df = pd.DataFrame({})
    merged_df = pd.DataFrame({})
    
    # obtain predictions over the dataframe
    for tile_name in tile_names:
        print("tile_name", tile_name)
        start_time = time.time()
        img_paths = glob(os.path.join(args.img_dir,"*"+tile_name+"*")) #identify the imgs correspondig to a given tile
        tile_path = os.path.join(args.tile_dir, tile_name +".tif") # specify the tile path
        #obtain tile information
        utmx, utmy, utm_proj, tile_band, tile_height, tile_width = tile_dimensions_and_utm_coords(tile_path) #used
        #predict on images
        predict_df_by_tank = predict_process(img_paths, tile_height, tile_width, model, args)
        predict_df_by_tank = copy(predict_df_by_tank[predict_df_by_tank.confidence > args.classification_threshold])
        #merge neighboring images
        merged_df_by_tank = merge_predicted_bboxes(predict_df_by_tank, dist_limit = 5)
        # calculate utm and lat lon coords
        
        merged_df_by_tank["utm_coords"] = merged_df_by_tank["bbox_pixel_coords"].apply(\
                                             lambda pixel_coords: get_utm_coords(pixel_coords, utmx, utmy))
        merged_df_by_tank["latlon_coords"] = merged_df_by_tank["utm_coords"].apply(\
                                             lambda utm_coords: get_lat_lon_coords(utm_coords, utm_proj))
        merged_df_by_tank["diameter"] = merged_df_by_tank["utm_coords"].apply(lambda utm_coord: calculate_diameter(utm_coord, resolution = 1))
        #specify the projection used 
        merged_df_by_tank["utm_proj"] = [utm_proj] * len(merged_df_by_tank)
      #update dataframes
        predict_df = pd.concat([predict_df, predict_df_by_tank], ignore_index=True)
        merged_df = pd.concat([merged_df, merged_df_by_tank], ignore_index=True)
        #delete temp dataframe to conserve memory
        del predict_df_by_tank, merged_df_by_tank
        end_time = time.time()
        execution_time = end_time - start_time
        print("Execution time:", execution_time, "seconds")   
        



tile_name m_3008929_ne_16_030_20211119

0: 640x640 (no detections), 1: 640x640 (no detections), 2: 640x640 (no detections), 3: 640x640 (no detections), 4: 640x640 (no detections), 5: 640x640 (no detections), 6: 640x640 (no detections), 7: 640x640 (no detections), 8: 640x640 (no detections), 9: 640x640 (no detections), 10: 640x640 (no detections), 11: 640x640 (no detections), 12: 640x640 (no detections), 13: 640x640 3 narrow_closed_roof_tanks, 14: 640x640 (no detections), 15: 640x640 (no detections), 16: 640x640 (no detections), 17: 640x640 (no detections), 18: 640x640 (no detections), 19: 640x640 (no detections), 20: 640x640 (no detections), 21: 640x640 (no detections), 22: 640x640 (no detections), 23: 640x640 1 external_floating_roof_tank, 1 undefined_object, 24: 640x640 (no detections), 25: 640x640 (no detections), 26: 640x640 (no detections), 27: 640x640 (no detections), 28: 640x640 (no detections), 29: 640x640 (no detections), 30: 640x640 1 narrow_closed_roof_tank, 31: 640x640 (no 

In [53]:
import os
import argparse
import time

import pystac_client
import planetary_computer
import stackstac

import geopandas as gpd
import numpy as np

import rasterio
from rasterio.merge import merge
from rasterio import mask
from rasterio import plot
import shapely
import ast


def read_raster(item_collection):
    #extract raster data
    raster_paths = [i.assets["data"].href for i in item_collection]
    rasters = [rasterio.open(raster_path) for raster_path in raster_paths]
    assert [raster.nodata == -9999 for raster in rasters]
    return rasters


def ensure_raster_tank_intersect(rasters, tank_geometry):
    raster_geoms = [shapely.box(raster.bounds.left,raster.bounds.bottom, raster.bounds.right, raster.bounds.top) for raster in rasters]
    raster_intersects = [tank_geometry.intersects(raster_geom) for raster_geom in raster_geoms]
    return np.array(rasters)[np.array(raster_intersects)]


def calculate_height(rasters, tank_geometry):
    # tank_geometry : geomtry of tank in utm
    #calculate the height for each raster
    h = []
    w = []
    for raster in rasters:
        clipped_image, clipped_transform = rasterio.mask.mask(raster, [tank_geometry], crop=True)
        #clipped_image.shape
        arr = np.array(clipped_image[clipped_image != -9999])
        if len(arr) > 0:
            h.append(np.quantile(arr.flatten(), 0.9))
            w.append(arr.size)
    [raster.close() for raster in rasters] #close rasters
    # average height
    if len(h) > 0:
        return np.average(h, weights=w)
    else:
        return None


def height_estimation(detected_tanks, collection):
    start = time.time()
    height_list = [] #height list
    catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1",
                                        modifier = planetary_computer.sign_inplace,)
    
    for tank_id, row in detected_tanks.iterrows():
        #create utm geometry

        tank_geometry = shapely.geometry.box(*row["utm_coords"], ccw=True) #utm
        #search catalog using lat lon geometry
        item_collection = catalog.search(collections=[collection], 
                                intersects=row.geometry.buffer(0.001)).item_collection()

        if len(item_collection) > 0:
            rasters = read_raster(item_collection)
            # ensure tank the data intersects
            rasters = ensure_raster_tank_intersect(rasters, tank_geometry)
            # calculate height
            height = calculate_height(rasters, tank_geometry)
            height_list.append(height)
        else:
            height_list.append(None) 
   
    print(time.time() - start)
    return height_list


def get_args_parse():
    parser = argparse.ArgumentParser("Height Estimation")
    parser.add_argument("--prediction_dir", default="/work/csr33/images_for_predictions/predictions", type=str)
    parser.add_argument("--collection", type=str, help="the name of the planetary computer collection")
    parser.add_argument("--chunk_id",  type=int)
    args = parser.parse_args()
    return args

In [18]:
import sys
sys.argv = ['my_notebook']
args = get_args_parse()
args.chunk_id =0

In [19]:
detected_tanks = gpd.read_parquet(os.path.join(args.prediction_dir, f"merged_predictions_{args.chunk_id}.parquet"))
detected_tanks['utm_coords'] = detected_tanks['utm_coords'].apply(lambda x: ast.literal_eval(x))

In [27]:
for tank_id, row in detected_tanks.iterrows():
    print(row.geometry.buffer(0.001))

POLYGON ((-89.43669843099575 30.607960250399383, -89.43660041385542 30.607955435126055, -89.43650334067374 30.607941035679787, -89.4364081463185 30.607917190735115, -89.43631574756338 30.607884129931893, -89.43622703425892 30.60784217166373, -89.43614286076273 30.607791720011683, -89.43606403771159 30.607733260852743, -89.43599132421456 30.607667357180567, -89.43592542054239 30.607594643683544, -89.43586696138344 30.6075158206324, -89.4358165097314 30.60743164713621, -89.43577455146324 30.60734293383175, -89.43574149066002 30.607250535076638, -89.43571764571534 30.6071553407214, -89.43570324626907 30.607058267539713, -89.43569843099574 30.606960250399382, -89.43569843099574 30.60694178634688, -89.43570324626907 30.60684376920655, -89.43571764571534 30.606746696024864, -89.43574149066002 30.606651501669624, -89.43577455146324 30.606559102914513, -89.4358165097314 30.606470389610053, -89.43586696138344 30.60638621611386, -89.43592542054239 30.606307393062718, -89.43599132421456 30.606234

In [35]:
collection="3dep-lidar-hag"

In [56]:
start = time.time()
height_list = [] #height list
catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1",
                                    modifier = planetary_computer.sign_inplace,)

In [57]:
for tank_id, row in detected_tanks.iterrows():
    #create utm geometry
    catalog.search(collections=[collection], 
                intersects=row.geometry)
    tank_geometry = shapely.geometry.box(*row["utm_coords"], ccw=True) #utm
    #search catalog using lat lon geometry
    item_collection = catalog.search(collections=[collection], 
                            intersects=row.geometry.buffer(0.001)).item_collection()

    if len(item_collection) > 0:
        rasters = read_raster(item_collection)
        # ensure tank the data intersects
        rasters = ensure_raster_tank_intersect(rasters, tank_geometry)
        # calculate height
        height = calculate_height(rasters, tank_geometry)
        height_list.append(height)
    else:
        height_list.append(None) 

34.900999307632446


In [45]:
import numpy as np

arr = np.array([1, 2, 3, -9999, 5, 6, -9999, 8, -9999, 10])

valid = arr[arr != -9999]

if len(valid) == 0:
    q90 = None
else:
    valid_sorted = np.sort(valid)
    index = int(0.9 * len(valid_sorted))
    q90 = valid_sorted[index]

print(q90)

10


In [22]:
height_estimation(detected_tanks, args.collection)


AttributeError: 'NoneType' object has no attribute 'id'

In [17]:
#reformat     


detected_tanks['utm_coords'] = detected_tanks['utm_coords'].apply(lambda x: str(x))
#detected_tanks.to_parquet(os.path.join(args.prediction_dir, f"merged_predictions_{args.chunk_id}.parquet"))

AttributeError: 'NoneType' object has no attribute 'id'

In [62]:
x = gpd.read_parquet(os.path.join(args.prediction_dir, f"merged_predictions_{1}.parquet"))
for h in x.height:
    print(h)

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
9.979999542236328
12.279999732971191
46.560001373291016
20.139999389648438
22.2720006942749
19.639999389648438
0.0
15.979999542236328
23.15999984741211
13.421999549865728
21.760000228881836
27.200000762939453
17.219999313354492
7.362000131607056
29.940000534057617
29.959999084472656
1.9600000381469727
10.84000015258789
17.3799991607666
15.821999740600585
15.621999931335449
5.320000171661377
5.380000114440918
15.720000267028809
15.600000381469727
16.959999084472656
17.59500026702881
17.210000038146973
26.781999397277833
15.720000267028809
14.760000228881836
14.760000228881836
14.779999732971191
14.779999732971191
67.71300277709975
14.859999656677246
21.692000579833984
19.299999237060547
18.059999465942383
19.219999313354492
0.0
15.680000305175781
15.859999656677246
13.739999771118164
16.1200008392334
15.183077225318321
15.619999885559082
15.619999885559082
15.846000289916992
16.040000915527344
22.100000381469727
20.7199